# AI-Based Trading Strategies

**Author:** J. Francisco Salazar

**Date:** 2025-02-23

## LSTM based Trading Strategy

# End-to-End Data Pipeline

### Template Notice (Editable Version)
> This notebook is a clean, reusable **template** for building and evaluating a stacked LSTM model on financial time series data.
>
> All key hyperparameters are defined in a centralized config block at the top of the notebook:
> 
> - Architecture: `LSTM_UNITS_1`, `LSTM_UNITS_2`, `DROPOUT_RATE`  
> - Training: `LEARNING_RATE`, `EPOCHS`, `BATCH_SIZE`  
> - Input shape: `SEQUENCE_LENGTH`, `N_FEATURES`  
>
> **To use this notebook:**
> - Edit the config block with your own values  
> - Then run all cells top-to-bottom  
> - Outputs are automatically generated and saved

This notebook is part of a modular pipeline. Each model is self-contained and organized for publishing, collaboration, and reproducibility.

## Model Configuration & Hyperparameters

> Set your custom model parameters here before running the notebook.  
> This block controls the LSTM architecture and training logic.  
> You must fill in values below or the notebook will not run.


In [ ]:
# ========================== CONFIG BLOCK ==========================
# ⚠️ Fill in your preferred values before running the notebook.

LSTM_UNITS_1 = None      # e.g., 128
LSTM_UNITS_2 = None      # e.g., 8
DROPOUT_RATE = None      # e.g., 0.2

LEARNING_RATE = None     # e.g., 0.0005
EPOCHS = None            # e.g., 30
BATCH_SIZE = None        # e.g., 64

SEQUENCE_LENGTH = None   # e.g., 7
N_FEATURES = None        # e.g., 15

# ================================================================

# Optional: fail-fast check
assert all(v is not None for v in [
    LSTM_UNITS_1, LSTM_UNITS_2, DROPOUT_RATE,
    LEARNING_RATE, EPOCHS, BATCH_SIZE,
    SEQUENCE_LENGTH, N_FEATURES
]), "⚠️ Please fill in all hyperparameter values above before running."


### Import Libraries & Dependencies
We import the necessary libraries for:
- API connection (`requests`)
- Data handling (`pandas`, `numpy`)
- Logging (`logging`)
- Visualization (`matplotlib`)


In [ ]:
# --- Standard Libraries ---
import os
import requests
import pandas as pd
import numpy as np
import logging
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta
from time import sleep

## API Configuration
- Load the API Key from environment variables.
- Define the ticker (`GSPC.INDX` for S&P 500).
- Set the date range (`2014-2025`).

In [ ]:
# --- API Configuration ---
API_KEY = os.getenv("EODHD_API_KEY")  # Load API Key securely
TICKER = "GSPC.INDX"
START_DATE = "2015-01-01"
END_DATE = "2025-01-01"
BASE_URL = "https://eodhd.com/api/eod/"

## API Request Function 
- Fetches historical data using the **EODHD API**.
- Implements **error handling & retry logic** to manage failed API calls.

In [ ]:
def fetch_eod_data(ticker, api_key, start_date, end_date, retries=3, delay=5):
    """
    Fetches historical market data from EODHD API with retry logic.

    Parameters:
    - ticker (str): Stock or index ticker symbol
    - api_key (str): API authentication token
    - start_date (str): Start date for data retrieval
    - end_date (str): End date for data retrieval
    - retries (int): Number of retry attempts in case of failure
    - delay (int): Delay between retries (exponential backoff)

    Returns:
    - pd.DataFrame: Market data if successful, else None
    """
    url = f"{BASE_URL}{ticker}?api_token={api_key}&from={start_date}&to={end_date}&fmt=json"
    
    for attempt in range(retries):
        try:
            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                data = response.json()
                if data:
                    logging.info(f"Successfully retrieved {len(data)} records.")
                    return pd.DataFrame(data)
                else:
                    logging.warning("API returned an empty dataset.")
            else:
                logging.error(f"API request failed with status {response.status_code}: {response.text}")
        except requests.RequestException as e:
            logging.error(f"API request error: {e}")

        sleep(delay * (2 ** attempt))  # Exponential backoff
    return None

## Fetch & Preprocess Data
- Convert `date` column to datetime format.
- Set `date` as index for time-series analysis.
- Convert numeric columns to `float`.
- Handle missing values using **forward-fill**.

In [ ]:
df = fetch_eod_data(TICKER, API_KEY, START_DATE, END_DATE)

if df is not None:
    
    df["date"] = pd.to_datetime(df["date"])
    df = df.sort_values("date").reset_index(drop=True)
    
    df.set_index("date", inplace=True)

    numeric_cols = ["open", "high", "low", "close", "adjusted_close", "volume"]
    df[numeric_cols] = df[numeric_cols].astype(float)

    df.ffill(inplace=True)  

    display(df.head())

## Data Visualization
- Plot **S&P 500 adjusted closing prices** over time.

In [ ]:
plt.style.use('seaborn-v0_8')

plt.figure(figsize=(12, 6))
plt.plot(df.index, df["adjusted_close"], linewidth=1.8, color='navy')

plt.title("S&P 500 Closing Prices (2014 - 2025)", fontsize=14)
plt.xlabel("Time", fontsize=12)
plt.ylabel("Closing Price (USD)", fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)

plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.gca().xaxis.set_major_locator(mdates.YearLocator(1))  
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y'))  

plt.show()

## Feature Engineering & Transformation

In [ ]:
# Compute log returns
df["log_returns"] = np.log(df["adjusted_close"] / df["adjusted_close"].shift(1))

### Volatility-Adjusted Returns:

In [ ]:
# Rolling Volatility (10-day)
df["volatility_10"] = df["log_returns"].rolling(10).std()

# Volatility-Adjusted Returns
df["vol_adj_returns"] = df["log_returns"] / df["volatility_10"]

df.dropna(inplace=True)

df.head()

### Technical Indicators  

LSTM models **learn patterns from sequences**, so we should select indicators that enhance **trend detection, volatility awareness, and momentum signals**.

| **Category**              | **Indicator**                                      | **Best Periods** | **Why Useful?**                                   |
|---------------------------|---------------------------------------------------|----------------|--------------------------------------------------|
| **Moving Averages**        | **EMA (Exponential Moving Avg)**                 | **EMA(50, 200)** | Captures recent trend changes faster           |
| **Momentum Indicators**    | **RSI (Relative Strength Index)**                | **RSI(14)** | Identifies momentum & overbought/oversold conditions   |
| **Momentum Indicators**    | **MACD (Moving Avg Convergence Divergence)**     | **MACD(12,26,9)** | Detects momentum shifts & trend reversals   |
| **Volatility Indicators**  | **ATR (Average True Range)**                     | **ATR(14)** | Measures market volatility & stop-loss adjustments        |
| **Trend Indicators**       | **SAR (Parabolic SAR)**                          | **SAR(Default)** | Tracks trend direction & reversals                        |
| **Trend Indicators**       | **SLOPE (Slope Indicator)**                      | **SLOPE(14)** | Confirms trend strength                        |
| **Trend Indicators**       | **ADX (Average Directional Index)**              | **ADX(14)** | Measures trend strength without direction |
| **Volume Indicators**      | **OBV (On-Balance Volume)**                      | **OBV(Default)** | Confirms trend strength via volume              |


In [ ]:
# Momentum Indicators: RSI & MACD
def compute_rsi(df, column="adjusted_close", period=14):
    delta = df[column].diff()
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)
    avg_gain = np.convolve(gain, np.ones(period)/period, mode='same')
    avg_loss = np.convolve(loss, np.ones(period)/period, mode='same')
    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs))

def compute_macd(df, column="adjusted_close", short_period=12, long_period=26, signal_period=9):
    short_ema = df[column].ewm(span=short_period, adjust=False).mean()
    long_ema = df[column].ewm(span=long_period, adjust=False).mean()
    macd_line = short_ema - long_ema
    signal_line = macd_line.ewm(span=signal_period, adjust=False).mean()
    return macd_line, signal_line

# Moving Averages: EMA 50 & EMA 200
def compute_ema(df, column="adjusted_close", period=50):
    return df[column].ewm(span=period, adjust=False).mean()

def compute_ema_200(df, column="adjusted_close", period=200):
    return df[column].ewm(span=period, adjust=False).mean()

# Volatility Indicator: ATR
def compute_atr(df, period=14):
    tr = np.maximum(df["high"] - df["low"],
                     np.maximum(abs(df["high"] - df["adjusted_close"].shift()),
                                abs(df["low"] - df["adjusted_close"].shift())))
    return tr.rolling(window=period).mean()

# Trend Indicators: SAR, Slope & ADX
def compute_slope(df, column="adjusted_close", period=14):
    return df[column].diff(period) / period

def compute_sar(df, acceleration=0.02, maximum=0.2):
    sar = np.zeros(len(df))
    trend = 1  # Assume first trend is bullish
    extreme_point = df["high"].iloc[0]
    af = acceleration
    
    for i in range(1, len(df)):
        sar[i] = sar[i - 1] + af * (extreme_point - sar[i - 1]) if trend == 1 else sar[i - 1] - af * (sar[i - 1] - extreme_point)
        extreme_point = max(extreme_point, df["high"].iloc[i]) if trend == 1 else min(extreme_point, df["low"].iloc[i])
        af = min(af + acceleration, maximum)
    return sar

def compute_adx(df, period=14):
    plus_dm = np.maximum(df["high"].diff(), 0)
    minus_dm = np.maximum(-df["low"].diff(), 0)
    tr = np.maximum(df["high"] - df["low"],
                     np.maximum(abs(df["high"] - df["adjusted_close"].shift()),
                                abs(df["low"] - df["adjusted_close"].shift())))
    plus_di = 100 * (plus_dm.rolling(window=period).mean() / tr.rolling(window=period).mean())
    minus_di = 100 * (minus_dm.rolling(window=period).mean() / tr.rolling(window=period).mean())
    dx = 100 * np.abs(plus_di - minus_di) / (plus_di + minus_di)
    return dx.rolling(window=period).mean()

# Volume Indicator: OBV
def compute_obv_vectorized(df):
    direction = np.sign(df["adjusted_close"].diff()).fillna(0)
    return (direction * df["volume"]).cumsum()


# Compute Indicators and Add to DataFrame
df["EMA_50"] = compute_ema(df, period=50)
df["EMA_200"] = compute_ema_200(df, period=200)
df["RSI_14"] = compute_rsi(df, period=14)
df["MACD"], df["MACD_Signal"] = compute_macd(df)
df["ATR_14"] = compute_atr(df, period=14)
df["SAR"] = compute_sar(df)
df["SLOPE_14"] = compute_slope(df, period=14)
df["ADX_14"] = compute_adx(df, period=14)
df["OBV"] = compute_obv_vectorized(df)

# Fill any NaNs from rolling calculations
df.fillna(0, inplace=True)

In [ ]:
df.tail()

### Regime Detection:
Use Hidden Markov Models (HMMs) to identify market regimes (bull/bear) and create regime-specific features.

In [ ]:
from hmmlearn.hmm import GaussianHMM
from sklearn.preprocessing import StandardScaler
from scipy.stats import mode

# =========================
# Regime Detection with HMM
# =========================

# --- Step 1: Define Number of Hidden States (Regimes) ---
n_states = 3  # 3 regimes: Bullish, Bearish, Neutral

# --- Step 2: Regime Detection with HMM ---
# Selecting relevant features for HMM training
hmm_features = ["log_returns", "volatility_10", "RSI_14", "MACD", "ATR_14", "OBV"]

df_hmm = df[hmm_features].dropna()  # Avoid missing values
X = df_hmm.values  

# --- Step 3: Normalize Features for HMM ---
scaler_hmm = StandardScaler()
X_hmm_scaled = scaler_hmm.fit_transform(X)  # Standardize only HMM features

# --- Step 4: Train HMM Model ---
hmm_model = GaussianHMM(n_components=n_states, covariance_type="full", n_iter=1000, random_state=42)
hmm_model.fit(X_hmm_scaled)

# --- Step 5: Predict Market Regimes ---
df.loc[df_hmm.index, "regime"] = hmm_model.predict(X_hmm_scaled)

# --- Step 6: Ensure all rows have a regime assigned ---
df["regime"] = df["regime"].ffill().bfill()  

# --- Step 7: Apply Smoothing to Reduce Noise in Regime Changes ---
def rolling_mode(series, window=7):
    def safe_mode(x):
        m = mode(x, keepdims=True)
        return m.mode[0] if len(m.mode) > 0 else x.iloc[-1]
    # FIX: Use strictly forward-looking rolling (center=False)
    return series.rolling(window, center=False).apply(safe_mode, raw=False)

df["regime_smoothed"] = rolling_mode(df["regime"], window=7).ffill().bfill().astype(int)

# --- Step 8: Mark Major Regime Change Points ---
# FIX: Remove lookahead by not using shift(-15)
df["regime_change"] = df["regime_smoothed"].diff().ne(0).astype(int)
# (Optional alternative: If you prefer to confirm a change after 15 days, you could delay labeling:
# df["regime_change"] = df["regime_smoothed"].diff().ne(0).shift(15).fillna(0).astype(int))

# --- Step 9: Creating Regime-Based Features ---
# To avoid mixing non-contiguous periods with the same regime label, we create a block id:
df["block_id"] = df["regime_smoothed"].ne(df["regime_smoothed"].shift()).cumsum()

def regime_sma(df, column, window):
    # Compute rolling average within each contiguous block only
    return df.groupby("block_id")[column].apply(lambda x: x.rolling(window, min_periods=1).mean()).reset_index(level=0, drop=True)

# Apply Regime-Based Features
df["SMA_50_regime"] = regime_sma(df, "adjusted_close", 50)
df["SMA_200_regime"] = regime_sma(df, "adjusted_close", 200)
df["volatility_10_regime"] = regime_sma(df, "volatility_10", 10)
df["RSI_14_regime"] = regime_sma(df, "RSI_14", 14)

df.dropna(inplace=True)  # Drop rows with missing values

# --- Step 10: Fixing Regime Mapping and Visualization ---
# Remap 0,1,2 → 1,2,3 for consistent labeling
df["regime_smoothed"] = df["regime_smoothed"].map({0: 1, 1: 2, 2: 3})

# Define color mapping and labels for visualization
regime_colors = {1: "red", 2: "green", 3: "blue"}
regime_labels = {1: "Regime 1", 2: "Regime 2", 3: "Regime 3"}

plt.figure(figsize=(12, 6))
# Plot regime-based adjusted close prices
for regime, color in regime_colors.items():
    plt.plot(df[df["regime_smoothed"] == regime].index,
             df[df["regime_smoothed"] == regime]["adjusted_close"],
             linestyle='-', marker='.', label=regime_labels[regime], color=color)
# Highlight regime change points
for change_date in df[df["regime_change"] == 1].index:
    plt.axvline(change_date, color='black', linestyle="--", alpha=0.5)
plt.title("Market Regimes Detected by HMM")
plt.xlabel("Date")
plt.ylabel("Adjusted Close Price")
plt.legend()
plt.tight_layout()
plt.savefig("1_regime_detection_chart.png", dpi=150, bbox_inches='tight')
plt.show()


# --- Step 11: Fixed Regime Frequency Distribution ---
plt.figure(figsize=(6, 4))
regime_counts = df["regime_smoothed"].value_counts().sort_index()
bars = plt.bar(regime_counts.index, regime_counts.values, color=[regime_colors[i] for i in regime_counts.index])
plt.xticks(ticks=regime_counts.index, labels=regime_counts.index)
plt.title("Regime Frequency Distribution")
plt.xlabel("Regime")
plt.ylabel("Count")
plt.xticks(rotation=0)
plt.tight_layout()
plt.savefig("2_regime_frequency_distribution.png", dpi=150, bbox_inches='tight')
plt.show()



In [ ]:
# --- Step 1: Ensure "regime_smoothed" Exists  ---
if "regime_smoothed" not in df.columns:
    print("Warning: 'regime_smoothed' column is missing! Recreating it...")
    if "regime" in df.columns:
        df["regime_smoothed"] = df["regime"].astype(int)
    else:
        raise KeyError("Error: Neither 'regime_smoothed' nor 'regime' found in DataFrame!")

# --- Step 2: Drop Any Old One-Hot Encoded Regime Columns (But NOT "regime_smoothed") ---
old_regime_cols = [col for col in df.columns if col.startswith("regime_") and col != "regime_smoothed"]
if old_regime_cols:
    df.drop(columns=old_regime_cols, inplace=True, errors="ignore")
    
# Preserve "regime_smoothed" before encoding to prevent KeyErrors
df["regime_smoothed_backup"] = df["regime_smoothed"]

# --- Step 3: One-Hot Encode Regimes for LSTM Training ---
df = pd.get_dummies(df, columns=["regime_smoothed"], prefix="regime")

# Ensure All Expected Regime Columns Exist (Fixes Missing regime_1)
expected_regimes = ["regime_1", "regime_2", "regime_3"]
for col in expected_regimes:
    if col not in df.columns:
        df[col] = 0  # Add missing regime column and fill with 0s

if "regime_smoothed" not in df.columns:
    df["regime_smoothed"] = df["regime_smoothed_backup"]
    
df.drop(columns=["regime_smoothed_backup"], inplace=True, errors="ignore")

# --- Step 4: Convert One-Hot Encoded Values to Integers ---
df[expected_regimes] = df[expected_regimes].astype(int)

# --- Step 5: Verify the Fix ---
display(df.tail())  # Check if columns are correct

print("Regime counts in dataset:")
print(df[expected_regimes].sum())  # Count number of rows for each regime

### Adding Lag Features (`lag_1` to `lag_7`) 

In [ ]:
# Generate lag features (up to 7 days)
lags = 7
for lag in range(1, lags + 1):
    df[f'lag_{lag}'] = df['log_returns'].shift(lag)

# Drop NaN values caused by shifting
df.dropna(inplace=True)

# Check DataFrame structure after adding lag features 
df.info()

### Feature Correlation Matrix 

In [ ]:
import seaborn as sns

corr_matrix = df.select_dtypes(include=[np.number]).corr()

# Generate a mask to hide the upper triangle (reduce redundancy)
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

# Dynamic figure sizing based on the number of features
n_features = corr_matrix.shape[0]
plt.figure(figsize=(n_features * 0.5, n_features * 0.5))  # Adjust multiplier if needed

sns.heatmap(
    corr_matrix, mask=mask, annot=False, fmt=".2f", cmap="coolwarm", center=0,
    linewidths=1, cbar_kws={"shrink": 0.75}, square=True
)

# Label formatting
plt.xticks(rotation=45, ha='right', fontsize=10)
plt.yticks(fontsize=10)
plt.title("Feature Correlation Matrix", fontsize=16, fontweight='bold', pad=20)

# Save figure (Change filename & format as needed)
plt.savefig("3_feature_correlation_matrix.png", dpi=300, bbox_inches='tight')

# Show the plot
plt.show()

### Definition of Target Variable (`d`)

In [ ]:
# --- Define Target Variable (`d`) ---
df['d'] = np.where(df['log_returns'].shift(-1) > 0, 1, 0)

# Drop only NaN values in 'd' caused by shifting (Last row)
df.dropna(subset=['d'], inplace=True)

# --- Corrected Feature Selection (X) ---
X = df[[col for col in [
    'open', 'high', 'low', 'close', 'adjusted_close', 'volume', 'log_returns', 
    'lag_1', 'lag_2', 'lag_3', 'lag_4', 'lag_5', 'lag_6', 'lag_7', 
    'volatility_10', 'vol_adj_returns', 'EMA_50', 'EMA_200',
    'RSI_14', 'MACD', 'MACD_Signal', 'ATR_14', 'SAR',
    'SLOPE_14', 'ADX_14', 'OBV', 'SMA_50_regime', 'SMA_200_regime', 
    'volatility_10_regime', 'RSI_14_regime', 'regime', 'regime_1', 'regime_2', 'regime_3'
] if col in df.columns]]  # Only include columns that exist

# --- Define Target Variable (y) ---
y = df['d']

# Verify Structure
print("Feature Matrix (X) Shape:", X.shape)
print("Target Variable (y) Shape:", y.shape)
display(df.tail())

In [ ]:
print(df['d'].value_counts(normalize=True))

### Feature Selection Using `ExtraTreesClassifier`

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1) Define a chronological cutoff date
train_cutoff = '2022-01-01'  # or pick another date

df_train = df.loc[:train_cutoff]
df_test = df.loc[train_cutoff:]

X_train = X.loc[df_train.index]
y_train = y.loc[df_train.index]
X_test = X.loc[df_test.index]
y_test = y.loc[df_test.index]

print("Training set shape:", X_train.shape, y_train.shape)
print("Test set shape:", X_test.shape, y_test.shape)

# 2) Fit ExtraTrees ONLY on the training set
model_fs = ExtraTreesClassifier(n_estimators=100, random_state=42)
model_fs.fit(X_train, y_train)

# 3) Extract feature importances from TRAINING
feature_importance = pd.Series(
    model_fs.feature_importances_,
    index=X_train.columns
).sort_values(ascending=False)

# (Optional) plot these feature importances
plt.figure(figsize=(12, 6))
sns.barplot(
    x=feature_importance.values, 
    y=feature_importance.index, 
    hue=feature_importance.index,
    palette="viridis",
    dodge=False,
    legend=False
)
plt.title("Feature Importance (ExtraTreesClassifier) - Training Set", fontsize=14, fontweight='bold')
plt.xlabel("Feature Importance Score")
plt.ylabel("Features")
plt.tight_layout()
plt.savefig("4_feature_importance_extratrees.png", dpi=100)  # or .pdf/.svg if needed
plt.show()

# 4) Select top N features from TRAINING
n_top = 15
top_features = feature_importance.index[:n_top]
print(f"Top {n_top} features (Training Set):", top_features.tolist())

# 5) Apply these features to both train & test
X_train_selected = X_train[top_features]
X_test_selected = X_test[top_features]

print("X_train_selected shape:", X_train_selected.shape)
print("X_test_selected shape:", X_test_selected.shape)


### Preparation of `X` and `y` for LSTM Model

In [ ]:
##############################
# Step 2: Scale Training Only
##############################
scaler = StandardScaler()
scaler.fit(X_train)            # Fit on training
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)  # Transform test with same scaler

# Convert back to DataFrame for convenience
X_train_scaled = pd.DataFrame(X_train_scaled, index=X_train.index, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, index=X_test.index, columns=X_test.columns)

##############################
# Step 3: Use Top Features
##############################
# 'top_features' is assumed to be known (selected from a training-only feature selection step)
# e.g., top_features = ['RSI_14', 'lag_6', 'lag_3', 'lag_5', 'vol_adj_returns', ...]
X_train_selected = X_train_scaled[top_features]
X_test_selected = X_test_scaled[top_features]

# Align target with these new DataFrame indices
y_train = y_train.loc[X_train_selected.index]
y_test = y_test.loc[X_test_selected.index]

print("X_train_selected shape:", X_train_selected.shape)
print("y_train shape:", y_train.shape)
print("X_test_selected shape:", X_test_selected.shape)
print("y_test shape:", y_test.shape)

##############################
# Step 4: Sequence Creation
##############################
def create_sequences(X_df, y_series, seq_length=7):
    """
    Generates sequences from a DataFrame (X_df) and aligns with the target (y_series).
    Returns (X_seq, y_seq).
    """
    X_values = X_df.values  
    y_values = y_series.values  
    
    # For each valid starting index i, the sequence is [i : i + seq_length]
    X_seq = np.array([
        X_values[i : i + seq_length] 
        for i in range(len(X_values) - seq_length)
    ])
    # The label for that sequence is y at index i+seq_length
    y_seq = y_values[seq_length:]  
    
    return X_seq, y_seq

sequence_length = 7

# Safety checks
if len(X_train_selected) < sequence_length:
    raise ValueError("Not enough training data to generate sequences! Adjust sequence_length or dataset.")
if len(X_test_selected) < sequence_length:
    raise ValueError("Not enough test data to generate sequences! Adjust sequence_length or dataset.")

# Generate training sequences
X_seq_train, y_seq_train = create_sequences(X_train_selected, y_train, seq_length=sequence_length)
# Generate test sequences
X_seq_test, y_seq_test = create_sequences(X_test_selected, y_test, seq_length=sequence_length)

##############################
# Step 5: Confirm Shapes
##############################
print("\nSequence Shapes:")
print(f"X_seq_train: {X_seq_train.shape} (Samples={X_seq_train.shape[0]}, Time Steps={sequence_length}, Features={X_seq_train.shape[2]})")
print(f"y_seq_train: {y_seq_train.shape} (Samples={y_seq_train.shape[0]})")
print(f"X_seq_test:  {X_seq_test.shape} (Samples={X_seq_test.shape[0]}, Time Steps={sequence_length}, Features={X_seq_test.shape[2]})")
print(f"y_seq_test:  {y_seq_test.shape} (Samples={y_seq_test.shape[0]})")

# Model Implementation & Training
## Model Definition & Initialization

#### LSTM Stacked Model

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping

# Model Definition & Initialization (uses CONFIG BLOCK values)

model = Sequential([
    Input(shape=(SEQUENCE_LENGTH, N_FEATURES)),  # Explicit Input layer

    # First LSTM layer: returns sequences for the next LSTM layer
    LSTM(LSTM_UNITS_1, return_sequences=True),
    Dropout(DROPOUT_RATE),

    # Second LSTM layer: processes the sequence and returns the final output
    LSTM(LSTM_UNITS_2, return_sequences=False),
    Dropout(DROPOUT_RATE),

    # Dense layer for binary classification
    Dense(1, activation='sigmoid')
])

# Compile the model with the specified learning rate
model.compile(
    loss='binary_crossentropy',
    optimizer=Adam(learning_rate=LEARNING_RATE),
    metrics=['accuracy']
)

# Early Stopping to avoid overfitting
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

# Display the model summary
model.summary()


## Model Training with Rolling Cross-Validation
### Training Implementation

* Rolling Time-Series Cross-Validation → Ensures robust evaluation.
* Scaling within each fold → Prevents data leakage.
* Early Stopping → Stops training if validation loss stops improving.
* Standard Metrics → Accuracy, Precision, Recall, F1-Score

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import pandas as pd

# ====================================================
# Function that pulls config params from the top block
# ====================================================
def get_stacked_model(input_shape):
    model = Sequential([
        Input(shape=input_shape),
        LSTM(LSTM_UNITS_1, return_sequences=True),
        Dropout(DROPOUT_RATE),
        LSTM(LSTM_UNITS_2, return_sequences=False),
        Dropout(DROPOUT_RATE),
        Dense(1, activation='sigmoid')
    ])

    model.compile(
        loss='binary_crossentropy',
        optimizer=Adam(learning_rate=LEARNING_RATE),
        metrics=['accuracy']
    )
    return model

# ====================================================
# Time-Series Cross-Validation Setup
# ====================================================
tscv = TimeSeriesSplit(n_splits=6, test_size=252)

predictions, true_labels = [], []
fold_metrics = []

for fold, (train_index, test_index) in enumerate(tscv.split(X_seq_train)):
    print(f"\nProcessing Fold {fold + 1}/{tscv.n_splits}...")

    # Extract fold data
    X_train_fold = X_seq_train[train_index]
    X_test_fold = X_seq_train[test_index]
    y_train_fold = y_seq_train[train_index]
    y_test_fold = y_seq_train[test_index]

    # Scale fold data (3D ➝ 2D ➝ 3D)
    nsamples_train, timesteps, n_feats = X_train_fold.shape
    X_train_fold_2d = X_train_fold.reshape(-1, n_feats)

    scaler = StandardScaler()
    X_train_fold_scaled = scaler.fit_transform(X_train_fold_2d).reshape(X_train_fold.shape)
    X_test_fold_scaled = scaler.transform(X_test_fold.reshape(-1, n_feats)).reshape(X_test_fold.shape)

    # Build and train model
    model = get_stacked_model((timesteps, n_feats))
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    model.fit(
        X_train_fold_scaled, y_train_fold,
        validation_data=(X_test_fold_scaled, y_test_fold),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        verbose=0,
        callbacks=[early_stopping]
    )

    # Predict and store results
    pred = model.predict(X_test_fold_scaled).flatten()
    pred_binary = [1 if p >= 0.5 else 0 for p in pred]

    predictions.extend(pred)
    true_labels.extend(y_test_fold)

    # Compute and store fold metrics
    fold_metrics.append({
        "fold": fold + 1,
        "accuracy": accuracy_score(y_test_fold, pred_binary),
        "precision": precision_score(y_test_fold, pred_binary),
        "recall": recall_score(y_test_fold, pred_binary),
        "f1_score": f1_score(y_test_fold, pred_binary)
    })

# ====================================================
# Save & Display Aggregate Metrics
# ====================================================
df_fold_metrics = pd.DataFrame(fold_metrics)
df_fold_metrics.to_csv("fold_metrics.csv", index=False)

print("\nFold Metrics:")
display(df_fold_metrics)

# Overall performance
predictions_binary = [1 if p >= 0.5 else 0 for p in predictions]
print("\nCross-Validated Model Performance (Training Set):")
print(f"Accuracy: {accuracy_score(true_labels, predictions_binary):.4f}")
print(f"Precision: {precision_score(true_labels, predictions_binary):.4f}")
print(f"Recall: {recall_score(true_labels, predictions_binary):.4f}")
print(f"F1-score: {f1_score(true_labels, predictions_binary):.4f}")

print("\nMean Fold Performance:")
print(df_fold_metrics[["accuracy", "precision", "recall", "f1_score"]].mean())


#### `Saving & Logging`

In [ ]:
import shutil 

# Save the best model from cross-validation
model.save("best_lstm_model.keras")

# Backup in case of overwriting
shutil.copy("best_lstm_model.keras", "best_lstm_model_backup.keras")

**Up to this section, we have put together a production-grade pipeline for an LSTM-based trading strategy.**  

# Trading Signal Generation and Backtesting Module

In [ ]:
def run_backtest(
    df, predictions, 
    transaction_cost=0.0001, 
    long_threshold=0.505, 
    short_threshold=0.495, 
    min_hold_days=3,
    stop_loss_threshold=-0.025,  # Stop-loss: Exit if daily loss > -2.5%
    rolling_loss_threshold=-0.05,  # Exit if rolling 10-day return < -5%
    volatility_scaling=True
):
    """
    Backtests a trading strategy with:
    - Lowered probability thresholds for better trade execution.
    - Adaptive holding period (exit when probability confidence weakens).
    - Stop-loss mechanism to prevent major drawdowns.
    - Volatility-based position scaling.
    
    Parameters:
        df (DataFrame): Historical price data including 'log_returns'.
        predictions (array-like): Model output probabilities for the test period.
        transaction_cost (float): Cost per trade.
        long_threshold (float): Minimum probability to enter a long trade.
        short_threshold (float): Maximum probability to enter a short trade.
        min_hold_days (int): Minimum days to hold a position once taken.
        stop_loss_threshold (float): Stop-loss exit threshold per trade.
        rolling_loss_threshold (float): Exit if rolling 10-day loss > threshold.
        volatility_scaling (bool): If True, position size scales based on volatility.

    Returns:
        df_backtest (DataFrame): DataFrame with backtest results including an equity curve.
    """
    df_backtest = df.copy()
    
    # Align predictions with the last len(predictions) rows of df_backtest
    df_backtest['p'] = np.nan
    df_backtest.loc[df_backtest.index[-len(predictions):], 'p'] = predictions

    # Generate raw trading signals based on adjusted thresholds
    df_backtest['raw_signal'] = np.select(
        [df_backtest['p'] >= long_threshold, df_backtest['p'] <= short_threshold],
        [1, -1], default=0
    )

    # ---------------------------
    # Adaptive Holding Period
    # ---------------------------
    raw_signals = df_backtest['raw_signal']
    final_positions = []
    current_position = 0
    hold_counter = 0

    for i in range(len(raw_signals)):
        if hold_counter > 0:
            # Continue holding the position
            final_positions.append(current_position)
            hold_counter -= 1
        else:
            # Allow new signal entry
            if raw_signals.iloc[i] != 0:
                current_position = raw_signals.iloc[i]
                hold_counter = min_hold_days - 1  # Hold for at least min_hold_days
                final_positions.append(current_position)
            else:
                # Exit if probability confidence weakens
                current_position = 0
                final_positions.append(0)

    df_backtest['po'] = final_positions
    df_backtest['po_shifted'] = df_backtest['po'].shift(1).fillna(0)  # Avoid look-ahead bias

    # ---------------------------
    # Stop-Loss Implementation
    # ---------------------------
    df_backtest['daily_return'] = df_backtest['po_shifted'] * df_backtest['log_returns']
    
    # Apply stop-loss: Exit position if loss exceeds stop_loss_threshold
    stop_loss_exit = df_backtest['daily_return'] <= stop_loss_threshold
    df_backtest.loc[stop_loss_exit, 'po_shifted'] = 0

    # Apply rolling 10-day loss exit condition
    df_backtest['rolling_loss'] = df_backtest['daily_return'].rolling(10).sum()
    df_backtest.loc[df_backtest['rolling_loss'] <= rolling_loss_threshold, 'po_shifted'] = 0

    # ---------------------------
    # Volatility-Based Position Sizing
    # ---------------------------
    if volatility_scaling:
        df_backtest['volatility_30d'] = df_backtest['log_returns'].rolling(30).std()
        avg_volatility = df_backtest['volatility_30d'].mean()
        df_backtest['position_size'] = df_backtest['po_shifted'] * (avg_volatility / df_backtest['volatility_30d'])
    else:
        df_backtest['position_size'] = df_backtest['po_shifted']

    # ---------------------------
    # Compute Strategy Returns
    # ---------------------------
    df_backtest['strategy_log_return'] = df_backtest['position_size'] * df_backtest['log_returns']

    # ---------------------------
    # Apply Transaction Costs
    # ---------------------------
    df_backtest['trade_cost'] = transaction_cost * df_backtest['po'].diff().abs().fillna(0)
    df_backtest['strategy_net_log_return'] = df_backtest['strategy_log_return'] - df_backtest['trade_cost']

    # ---------------------------
    # Compute Equity Curve
    # ---------------------------
    df_backtest['equity_curve'] = np.exp(df_backtest['strategy_net_log_return'].cumsum())

    return df_backtest

#  Run the backtest
df_backtest = run_backtest(df, predictions)

#  Print results
print(df_backtest[['log_returns', 'p', 'po', 'strategy_log_return', 'trade_cost', 'strategy_net_log_return', 'equity_curve']].tail())


# AI Strategy Performance Review

## Plot 1: Actual Returns vs Predicted Returns
Shows how well the signals align with actual movements.

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(df_backtest.index, df_backtest['log_returns'], label="Actual Returns", alpha=0.45)
plt.plot(df_backtest.index, df_backtest['strategy_log_return'], label="Predicted Returns", alpha=0.45)
plt.axhline(y=0, color='gray', linestyle='--', linewidth=0.8)
plt.title("Actual Returns vs Predicted Returns")
plt.ylabel("Daily Log Returns")
plt.legend()
# Save the figure
plt.tight_layout()
plt.savefig("5_actual_vs_predicted_returns.png", dpi=100)
plt.show()


## Plot 2: Equity Curve of the Strategy
Show the cumulative performance over time.

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df_backtest.index, df_backtest['equity_curve'], label="Strategy Equity Curve", color='blue')
plt.axhline(1.0, linestyle='--', color='red', label="Baseline")
plt.xlabel("Date")
plt.ylabel("Cumulative Returns")
plt.title("Trading Strategy Backtest: Equity Curve")
plt.legend()
plt.grid()
plt.tight_layout()
plt.savefig("6_equity_curve_strategy.png", dpi=100)
plt.show()


## Plot 3: Strategy vs. Benchmark Comparison
Illustrate whether the strategy outperforms a simple buy-and-hold benchmark.

In [ ]:
# Convert log returns to cumulative returns for buy-and-hold:
df_backtest['buy_and_hold'] = np.exp(df_backtest['log_returns'].cumsum())
df_backtest['cumulative_strategy'] = df_backtest['equity_curve']

plt.figure(figsize=(10, 6))
plt.plot(df_backtest.index, df_backtest['buy_and_hold'], label="Buy-and-Hold (Benchmark)", alpha=0.55)
plt.plot(df_backtest.index, df_backtest['cumulative_strategy'], label="LSTM Strategy", alpha=0.55)
plt.axhline(y=1, color='gray', linestyle='--', linewidth=0.8)
plt.title("Cumulative Performance: Strategy vs. Benchmark")
plt.ylabel("Cumulative Returns")
plt.legend()
plt.tight_layout()
plt.savefig("7_cumulative_performance_vs_benchmark.png", dpi=300)

plt.show()


## `quantstats` Analytics Module


In [ ]:
# Convert net log returns to simple daily returns:
strategy_simple_returns = np.exp(df_backtest['strategy_net_log_return']) - 1
benchmark_simple_returns = np.exp(df_backtest['log_returns']) - 1

import quantstats as qs

# Generate a Performance Report
qs.reports.basic(strategy_simple_returns, benchmark=benchmark_simple_returns)

# Cumulative Returns Comparison (Strategy vs. Benchmark)
qs.plots.returns(strategy_simple_returns, benchmark=benchmark_simple_returns)

# Drawdown Analysis
qs.plots.drawdown(strategy_simple_returns)

# Rolling Sharpe Ratio
qs.plots.rolling_sharpe(strategy_simple_returns)


In [ ]:
# === Export Clean LSTM Model Results (Fixed) ===

df_lstm = df_backtest[['log_returns', 'p']].copy()
df_lstm.rename(columns={'p': 'predictions'}, inplace=True)

# Drop rows with missing predictions (common in time series)
df_lstm.dropna(subset=['predictions'], inplace=True)

# Confirm structure
print(df_lstm.head())
print(df_lstm.shape)


In [ ]:
df_lstm.to_csv('df_lstm.csv', index=False)